In [24]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import WordNetLemmatizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
message = pd.read_csv('/content/SMSSpamCollection',sep='\t',names=['labels','messages'])

In [10]:
message

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


import numpy as np
from google.colab import autoviz
df_3659860408095419855 = autoviz.get_registered_df('df_3659860408095419855')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_3659860408095419855, *['labels'], **{})
chart

In [26]:
#Text preprossing
corpus=[]
for msg in message['messages']:
  #removing special chars using regix and make all words to lowercase
  review = re.sub('[^a-zA-Z]',' ',msg)
  review = review.lower()
  #Word Tokenization
  words = nltk.word_tokenize(review)
  #Remove all the stopwords from the list by taking only meaningful words
  lem = WordNetLemmatizer()
  words = [lem.lemmatize(word) for word in words if word not in stopwords.words('english')]
  sent = ' '.join(words)
  corpus.append(sent)


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(corpus).toarray()

X = vector
y = message['labels']

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
vector = CV.fit_transform(corpus).toarray()

X = vector
y = message['labels']

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [37]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       955
        spam       0.98      0.76      0.86       160

    accuracy                           0.96      1115
   macro avg       0.97      0.88      0.92      1115
weighted avg       0.96      0.96      0.96      1115



In [38]:
nb = MultinomialNB()
nb.fit(X_train,y_train)
print(classification_report(y_test,nb.predict(X_test)))

              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       955
        spam       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [39]:
de = DecisionTreeClassifier()
de.fit(X_train,y_train)
print(classification_report(y_test,de.predict(X_test)))

              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       955
        spam       0.87      0.91      0.89       160

    accuracy                           0.97      1115
   macro avg       0.93      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [40]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train,y_train)
print(classification_report(y_test,rf.predict(X_test)))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       955
        spam       1.00      0.87      0.93       160

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



We Can see the Best F1 score and Accuracy is 90%+ for ***Random Forest***